### Test

In [9]:
import torch
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2Model

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2Model.from_pretrained("Salesforce/blip2-opt-2.7b")
prompt = "Question: how many cats are there? Answer:"

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(images=image, text=prompt, return_tensors="pt")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
qformer_outputs = model(**inputs)

In [19]:
for key in qformer_outputs: print(key)

logits
vision_outputs
qformer_outputs
language_model_outputs


In [ ]:
for key in qformer_outputs['qformer_outputs']['last_hidden_state']: print(key.shape)

In [24]:
qformer_outputs['qformer_outputs']['pooler_output'].shape

torch.Size([1, 768])

In [23]:
qformer_outputs['vision_outputs']['pooler_output']

tensor([[ 0.2400,  0.4323,  1.2008,  ..., -1.0685, -0.5133,  0.4215]],
       grad_fn=<NativeLayerNormBackward0>)

In [48]:
from PIL import Image
import requests
from transformers import AutoProcessor, LlavaForConditionalGeneration

model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf")
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

prompt = "<image>\nUSER: What's the content of the image?\nASSISTANT:"
url = "https://www.ilankelman.org/stopsigns/australia.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=[prompt, prompt], images=[image, image], return_tensors="pt", truncation=True, max_length=150)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [49]:
inputs['input_ids'].shape

torch.Size([2, 20])

In [45]:
# Generate
generate_ids = model(**inputs, output_hidden_states=True)

In [41]:
generate_ids['hidden_states'][-1].mean(dim=1).shape

torch.Size([2, 4096])

In [31]:
generate_ids[1].shape

torch.Size([2, 595, 32064])

In [32]:
for key in generate_ids: print(key)

loss
logits
past_key_values
hidden_states


In [35]:
for key in generate_ids['hidden_states']: print(key.mean(dim=1).shape)

torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
torch.Size([2, 4096])
